# Imports

In [1]:
import numpy as np
from os import listdir
from os.path import isfile
from os.path import isdir
from os.path import join
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import shutil
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import copy
import random
import time

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold

from torch.utils.data import TensorDataset, DataLoader

import wandb

Misc functions

In [2]:
def load_data(path):
  X = []
  Y = []

  onlyDirs = [f for f in listdir(path) if isdir(join(path, f))]

  for category in onlyDirs:
    pos_y = onlyDirs.index(category)
    print(category, pos_y)
    tempDirInput = join(path, category)
    for f in listdir(tempDirInput):
      inputPath = (join(tempDirInput, f))
      if isfile(inputPath):
        X.append(np.load(inputPath))
        one_hot = np.zeros((len(onlyDirs)))
        one_hot[pos_y] = 1
        Y.append(one_hot)

  return X, Y

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
def calculate_accuracy(y_pred, y):
    
    top_pred = y_pred.argmax(1, keepdim = True)
    
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

Paths dir

In [5]:
# Folder paths
histOutputDir = 'data/pp/hist'

run_ID = 2

# Load training and testing set

In [6]:
X_test, y_test = load_data(os.path.join(histOutputDir,'test'))
X_train, y_train = load_data(os.path.join(histOutputDir,'train'))

COVID 0
Lung_Opacity 1
Normal 2
Viral Pneumonia 3
COVID 0
Lung_Opacity 1
Normal 2
Viral Pneumonia 3


Global parameters

In [7]:
configwand = {
    'epochs'        :   600,
    'learning_rate' :   0.0001,
    'batch_size'    :   128,
    'input_dim'     :   256,
    'fc_size_1'     :   256,
    'fc_size_2'     :   128,
    'fc_size_3'     :   64,
    'fc_size_4'     :   32,
    'fc_size_5'     :   16,
    'fc_size_6'     :   8,
    'output_dim'    :   4
}

## Create a Data Loader

In [8]:
tensor_x = torch.Tensor(X_train) 
tensor_y = torch.Tensor(y_train)
my_dataset = TensorDataset(tensor_x,tensor_y) 
trainingSetLoader = DataLoader(my_dataset,
        shuffle=True,
        batch_size=configwand['batch_size']) 

tensor_x_test = torch.Tensor(X_test) 
tensor_y_test = torch.Tensor(y_test)
my_dataset_test = TensorDataset(tensor_x_test,tensor_y_test) 
testSetLoader = DataLoader(my_dataset_test,
        shuffle=True,
        batch_size=configwand['batch_size']) 


## Model

In [9]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, configwand['fc_size_1']),
            nn.ReLU(),
            nn.Linear(configwand['fc_size_1'], configwand['fc_size_2']),
            nn.ReLU(),
            nn.Linear(configwand['fc_size_2'], configwand['fc_size_3']),
            nn.ReLU(),
            nn.Linear(configwand['fc_size_3'], configwand['fc_size_4']),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(configwand['fc_size_4'], configwand['fc_size_5']),
            nn.ReLU(),
            nn.Linear(configwand['fc_size_5'], configwand['fc_size_6']),
            nn.ReLU(),
            nn.Linear(configwand['fc_size_6'], output_dim),
            nn.Softmax()
            )
    
    def forward(self, x):
        return self.layers(x)
    

In [10]:
def trainAndEvaluate(model, optimizer, criterion):
    for epoch in range(0, configwand['epochs']):
        is_training = True
        with torch.set_grad_enabled(is_training):
            model.train()
            running_loss = 0.0
            running_acc = 0.0
            for i_Train, (x, y) in enumerate(trainingSetLoader):
                
                optimizer.zero_grad()
                y_pred = model(x)
                y_argmax = torch.empty(y.shape[0], dtype=torch.long)
                y_argmax = torch.argmax(y, dim=1)
                loss = criterion(y_pred, y_argmax)
                
                acc = calculate_accuracy(y_pred, y_argmax)
                loss.backward()
        
                running_loss += loss.item()
                running_acc += acc

                optimizer.step()

        model.eval()
        is_training = False

        running_test_loss = 0.0
        running_test_acc = 0.0

        with torch.set_grad_enabled(is_training):
            for i_Test, (x, y) in enumerate(testSetLoader):
                
                y_pred = model(x)
                y_argmax = torch.empty(y.shape[0], dtype=torch.long)
                y_argmax = torch.argmax(y, dim=1)

                loss = criterion(y_pred, y_argmax)
                acc = calculate_accuracy(y_pred, y_argmax)
        
                running_test_loss += loss.item()
                running_test_acc += acc

        
        wandb.log({"Training Loss"       :  running_loss/i_Train,
                    "Training Accuracy"   :  running_acc/i_Train})

        wandb.log({"Test Loss"       :  running_test_loss/i_Test,
                    "Test Accuracy"   :  running_test_acc/i_Test})

        if(epoch%20 == 0):
            print("Epoch %d, training loss: {%.3f}, training acc {%.3f}" % (epoch, 
                    running_loss/i_Train, running_acc/i_Train))
            print("Epoch %d, testing loss: {%.3f}, testing acc {%.3f}" % (epoch, 
                    running_test_loss/i_Test, running_test_acc/i_Test))
            print('******************************************')
            
model = MLP(configwand['input_dim'], configwand['output_dim'])
optimizer = optim.Adam(model.parameters(),lr=configwand['learning_rate'])
criterion = nn.CrossEntropyLoss()
trainAndEvaluate(model,optimizer,criterion)

# Run

In [11]:
run_ID += 1
print(run_ID)

3


In [12]:
#Create the model, optimizer and criterion
run_ID += 1
model = MLP(configwand['input_dim'], configwand['output_dim'])
optimizer = optim.Adam(model.parameters(),lr=configwand['learning_rate'])
criterion = nn.CrossEntropyLoss()

#Init WandB
name = "MLP "+ str(run_ID)
run = wandb.init(project='MLP', entity='tecai', config=configwand,
                        name=name)
wandb.watch(model)
#Train and evaluate the model
trainAndEvaluate(model,optimizer,criterion)
#Finish WandB
run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stevenpach10 (use `wandb login --relogin` to force relogin)
C:\Users\Steven\AppData\Roaming\Python\Python39\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


C:\Python39\lib\site-packages\torch\nn\modules\container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
C:\Python39\lib\site-packages\torch\autograd\__init__.py:145: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(
Epoch 0, training loss: {1.371}, training acc {0.482}
Epoch 0, testing loss: {1.398}, testing acc {0.515}
******************************************
Epoch 20, training loss: {1.197}, training acc {0.549}
Epoch 20, testing loss: {1.210}, testing acc {0.573}
******************************************
Epoch 40, training loss: {1.162}, training acc {0.595}
Epoch 40, testing loss: {1.180}, testing acc {0.613}
******************************************
Epoch 60, training loss: {1.143}, training acc {0.614}
Epoch 60, testing

Training Loss,1.055
Training Accuracy,0.70061
_runtime,920
_timestamp,1620331924
_step,1199
Test Loss,1.09571
Test Accuracy,0.70007


Training Loss,█▇▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▁▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Test Loss,█▇▅▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▁▁▁▂▁▂▁▁▁▁
Test Accuracy,▁▂▄▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███▇▇█████▇█▇████
